<a href="https://colab.research.google.com/github/litAardvark/CSE151A_Group_Project_SII25/blob/main/CSE151A_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Abstract
Abstract (Work in progress):

Flight delays are a costly and frustrating inevitability for even the savviest of flyers. Itineraries that accurately account for delays stand to ease a major pain point for travelers on commercial airlines. Our project seeks to address this problem by predicting the length of flight delays using machine learning. To this end, we propose the use of a supervised Support Vector Regression model. Our proposed dataset contains flight data from 2009 to 2019, gathered from the US Department of Transportation's Bureau of Transportation Statistics. We plan to implement a machine learning pipeline for training and testing. The output of our final model will enable us to identify the variables associated with longer delays.

# Datasets

**Dataset #1**: Airline Delay Analysis - contains relevant information about most US flights for a period of ten years beginning in 2009 and ending in 2019.

Link: https://www.kaggle.com/datasets/sherrytp/airline-delay-analysis

Location: /content/airline_delay_analysis


# Team

Jason Bulux
jbulux@ucsd.edu

Rosario Ortiz
mdortiz@ucsd.edu

Kwadwo Danquah
kdanquah@ucsd.edu

Sam Kogan
skogan@ucsd.edu

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Note: Download your kaggle API key and save it on your Google Drive at the path:
MyDrive/Colab Notebooks

In [ ]:
!mkdir ~/.kaggle
!cp '/content/drive/MyDrive/Colab Notebooks/kaggle.json' ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!kaggle datasets download -d sherrytp/airline-delay-analysis

Dataset URL: https://www.kaggle.com/datasets/sherrytp/airline-delay-analysis
License(s): DbCL-1.0
 99% 2.17G/2.19G [00:31<00:00, 103MB/s] 
100% 2.19G/2.19G [00:31<00:00, 74.1MB/s]


In [ ]:
!unzip airline-delay-analysis.zip

Archive:  airline-delay-analysis.zip
replace airline delay analysis/20.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 